# GDA Implementation.

Implement the Gaussian Discriminant Analysis (GDA) learning algorithm following the steps as discussed in class.

INSTRUCTION: Rename your notebook as: <br>
`firstName_LastName_Live_coding_GDA.ipynb`.

Notes: 
* Do not use any built-in functions to complete a task;
* Do not import additional libraries.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification

In [3]:
# Generate data
def generate_data():
  x, y = make_classification(n_samples= 1000, n_features=3, n_redundant=0, 
                           n_informative=3, random_state=1, 
                           n_clusters_per_class=1)
  
  return x,y

x,y= generate_data()
print(x.shape, y.shape)

(1000, 3) (1000,)


In [114]:
def split_data(x,y, train_size= 0.8):
    # shuffle the data to randomize the train/test split
    indice = np.arange(x.shape[0])
    np.random.permutation(indice)
    x= x[indice]
    y = y[indice]
    num = int(train_size * x.shape[0])
    x_train = x[:num]
    y_train = y[:num]
    x_test = x[num:]
    y_test = y[num:]
    return x_train, x_test, y_train, y_test


In [173]:
x_train, x_test, y_train, y_test= split_data(x,y, train_size= 0.8) # split your data into x_train, x_test, y_train, y_test
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(800, 3) (800,) (200, 3) (200,)


In [188]:
def covariance(x, mu):

  # Easy way: cov= np.cov(x, rowvar=0) but do not use it. One can use it to assess his/her result.
  N,d = x.shape
  sigma = np.zeros((d,d))
  K=mu.shape[0]
  for k in range(K):
    for i in range(d):
      for j in range(d):
        sigma[i,j] = 1/(N-1)*np.sum((x[:,i] - mu[i]) * (x[:,j] - mu[j])) 
  return sigma

In [189]:
covariance(x, x.mean(axis=0))

array([[1.84495325, 0.02790646, 1.00137533],
       [0.02790646, 1.00170721, 0.05539176],
       [1.00137533, 0.05539176, 1.74832   ]])

In [69]:
toy_x = np.random.randn(10,3)
toy_mu = np.mean(toy_x, 0)
print(toy_x.shape, toy_mu.shape)

(10, 3) (3,)


In [176]:
np.cov(toy_x, rowvar=0)

array([[ 0.42578484,  0.16536629, -0.36401319],
       [ 0.16536629,  1.97368177, -0.4074186 ],
       [-0.36401319, -0.4074186 ,  2.10051637]])

In [274]:
from numpy.core.fromnumeric import argmax
from math import pi
from numpy.random import Philox
class GDA:
  def __init__(self):
    ## set mu, phi and sigma to None
    self.mu = None
    self.phi =None
    self.sigma = None

  def fit(self,x,y):
    k=np.unique(y).shape[0] # Number of class.
    d=x.shape[1]  # input dim
    m=x.shape[0]  # Number of examples.
    
    ## Initialize mu, phi and sigma
    self.mu= np.zeros((k,d))#: kxd, i.e., each row contains an individual class mu.
    self.sigma= np.zeros((k,d,d))#: kxdxd, i.e., each row contains an individual class sigma.
    self.phi= np.zeros(k)# d-dimension
    for label in range(k):
      self.phi[label] = np.sum(label == y)/x.shape[0]
      #idx=np.where(y==label)
      self.mu[label] = np.mean(x[label == y], axis=0)
      self.sigma[label] = covariance(x[label == y], self.mu[label])

    ## START THE LEARNING: estimate mu, phi and sigma.
    




  def predict_proba(self,x):
    # reshape or flatt x.
    #x= x.reshape[0]
    d=x.shape[1]
    k_class= self.mu.shape[0]# Number of classes we have in our case it's k = 2
    print(x.shape[0])
    ## START THE LEARNING: estimate mu, phi and sigma.
    #mu = self.mu
    #phi =self.phi
    #sigma = self.sigma
    pro = np.zeros((x.shape[0], k_class))
    
    for lab in range(k_class):
      for i in range(x.shape[0]):
        coeff = 1/((2*pi)**(d/2)) *(np.linalg.det(self.sigma[lab])**(1/2))
        pro[i,lab] = coeff * np.exp(-(1/2)*(x[i] - self.mu[lab]).T@np.linalg.inv(self.sigma[lab])@(x[i] - self.mu[lab]))
    return pro



  def predict(self,x):
    predict_proba = self.predict_proba(x)*self.phi
    y_pred = np.argmax(predict_proba, axis = 1)
    return y_pred
  
  def accuracy(self, y, ypreds):
    accu = np.mean(y==ypreds)
    return accu

In [275]:
model= GDA()
model.fit(x_train,y_train)

In [276]:
model.mu

array([[ 1.00201779,  1.00494482,  0.99548846],
       [-1.01641736,  0.96824826, -0.90433708]])

In [277]:
model.phi

array([0.495, 0.505])

In [278]:
model.sigma

array([[[ 0.90164773, -0.43335029, -0.06500866],
        [-0.43335029,  1.68747351,  0.10027055],
        [-0.06500866,  0.10027055,  0.0384381 ]],

       [[ 0.7647548 ,  0.35278165,  0.0911074 ],
        [ 0.35278165,  0.34171233, -0.08598807],
        [ 0.0911074 , -0.08598807,  1.62087217]]])

In [279]:
yproba = model.predict_proba(x_test)
yproba.shape

200


(200, 2)

In [280]:
ypreds= model.predict(x_test)
ypreds


200


array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 0])

In [281]:
model.accuracy(y_test, ypreds)

0.965